# OFFLINE PATH PLANNING OPTIMISATION
This notebook elaborates the testing and development of an offline path planning optimisation pipeline to generate a safe and feasible reference path for a ego vehicle. The pipeline takes a set of coarsely placed waypoints and adjusts their localisation so as to comply with certain path curvature constraints in view of the vehicle's size and steering capability. The adjusted waypoints should be as close as possible to the original waypoints without violating the curvature constraints.

### Installation
Before beginning, it is important to install all the dependencies to run all cells in this notebook.

!pip install -r requirements.txt

### Imports

In [1]:
import numpy as np
import pandas as pd
import scipy.interpolate as sp_interp
import scipy.optimize as optimize

from numba import njit

### Generating Waypoints
To easily generate waypoints for testing, we will use a modified version of my [Waypoint Generator](https://github.com/winstxnhdw/WaypointGenerator) script. The generated waypoints are exported as a CSV file and can be imported again using the pandas library.

In [2]:
%matplotlib widget

import matplotlib.pyplot as plt

def onclick(event):

    x.append(event.xdata)
    y.append(event.ydata)

    ax.plot(x, y, '-', color=line_colour)
    ax.plot(x, y, '.', color=point_colour)

    fig.canvas.draw()

    axis = {'X-axis': x, 'Y-axis': y}
    df = pd.DataFrame(axis, columns= ['X-axis', 'Y-axis'])
    df.to_csv("waypoints.csv", index = False)

def onpress(event):

    # Undo last point
    if event.key == 'z':
        try:
            x.pop()
            y.pop()

        except:
            pass
    
    # Clear all the points
    elif event.key == 'x':
        del x[:]
        del y[:]

    # Connect the first and last points
    elif event.key == 'c':
        try:
            x.append(x[0])
            y.append(y[0])

        except:
            pass

    else:
        pass

    plt.cla()
    plt.grid()
    ax.set_xlim(-map_size, map_size)
    ax.set_ylim(-map_size, map_size)
    ax.plot(x, y, '-', color=line_colour)
    ax.plot(x, y, '.', color=point_colour)
    fig.canvas.draw()

    axis = {'X-axis': x, 'Y-axis': y}
    df = pd.DataFrame(axis, columns= ['X-axis', 'Y-axis'])
    df.to_csv("waypoints.csv", index = False)

# Parameters
map_size = 100
line_colour = '#F0A39A'
point_colour = '#383831'
x = []
y = []

fig = plt.figure()
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.resizable = True
fig.canvas.mpl_connect('button_press_event', onclick)
fig.canvas.mpl_connect('key_press_event', onpress)

ax = plt.axes()
ax.set_aspect('equal', adjustable='box')
ax.set_xlim(-map_size, map_size)
ax.set_ylim(-map_size, map_size)

plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Export Notebook as Markdown

!jupyter nbconvert main.ipynb --to markdown --output README.md